# Google Gemini: Structured Output with JSON Schema

This notebook demonstrates how to use JSON Schema to define the expected structure for responses from the Gemini API. It covers various JSON Schema features, including nested objects, arrays, enums, default values, patterns, and union types. We will also explore how to represent a similar schema using Pydantic and generate a JSON Schema from it.

In [ ]:
%pip install google-genai pydantic

In [ ]:
import os
import json
from google import genai
from pydantic import BaseModel, Field, RootModel, EmailStr
from typing import Union, Optional, List, Literal

# Configure the client
# Make sure to set your GEMINI_API_KEY environment variable
# or pass it directly: genai.Client(api_key="YOUR_API_KEY")
try:
    client = genai.Client() 
except Exception as e:
    print(f"Error initializing Gemini client: {e}. Please ensure your API key is set correctly.")
    client = None 
    # Add further handling if client initialization is critical for subsequent cells

## Using JSON Schema for Structured Output

Besides Pydantic, you can also define the response schema using JSON Schema directly. This provides a flexible way to specify the expected structure of the JSON output. The following example demonstrates various features of JSON Schema, including nested objects, arrays, enums, default values, and union types.

In [ ]:
my_schema = {
    "type": "object",
    "title": "ProductCatalog",
    "description": "Schema for a product catalog item.",
    "properties": {
        "product_id": {
            "type": "string",
            "description": "Unique identifier for the product.",
            "pattern": "^[A-Z0-9]{5,10}$"
        },
        "product_name": {
            "type": "string",
            "description": "Name of the product."
        },
        "price": {
            "type": "number",
            "description": "Price of the product.",
            "minimum": 0.01,
            "maximum": 10000.00
        },
        "in_stock": {
            "type": "boolean",
            "description": "Availability status of the product.",
            "default": True
        },
        "tags": {
            "type": "array",
            "description": "Keywords for the product.",
            "items": {
                "type": "string"
            },
            "minItems": 1,
            "maxItems": 5
        },
        "category": {
            "type": "string",
            "description": "Product category.",
            "enum": ["Electronics", "Apparel", "Home Goods", "Books"]
        },
        "dimensions_or_weight": {
            "anyOf": [
                { "$ref": "#/$defs/dimensions" },
                { "$ref": "#/$defs/weight" }
            ],
            "description": "Either physical dimensions or weight of the product."
        },
        "supplier": {
            "$ref": "#/$defs/supplierInfo"
        }
    },
    "required": ["product_id", "product_name", "price", "category"],
    "$defs": {
        "dimensions": {
            "type": "object",
            "title": "Dimensions",
            "properties": {
                "length": {"type": "number", "description": "Length in cm."},
                "width": {"type": "number", "description": "Width in cm."},
                "height": {"type": "number", "description": "Height in cm."}
            },
            "required": ["length", "width", "height"]
        },
        "weight": {
            "type": "object",
            "title": "Weight",
            "properties": {
                "value": {"type": "number", "description": "Weight value."},
                "unit": {"type": "string", "enum": ["kg", "lb"], "default": "kg"}
            },
            "required": ["value", "unit"]
        },
        "supplierInfo": {
            "type": "object",
            "title": "SupplierInformation",
            "description": "Information about the product supplier.",
            "properties": {
                "supplier_name": {"type": "string"},
                "contact_email": {"type": "string", "format": "email"}
            },
            "required": ["supplier_name"]
        }
    }
}

# You can optionally print the schema to verify its structure
# import json
# print(json.dumps(my_schema, indent=2))

## Generating JSON Schema from Pydantic Models

Pydantic models can also be used to generate a JSON Schema. This is often a more convenient way to define complex schemas, as you can leverage Python's type system and Pydantic's validation capabilities. Below, we define Pydantic models equivalent to the `my_schema` defined earlier.

In [ ]:
from pydantic import BaseModel, Field, RootModel, EmailStr
from typing import Union, Optional, List, Literal

class Dimensions(BaseModel):
    length: float = Field(description="Length in cm.")
    width: float = Field(description="Width in cm.")
    height: float = Field(description="Height in cm.")

class Weight(BaseModel):
    value: float = Field(description="Weight value.")
    unit: Literal["kg", "lb"] = Field(default="kg", description="Weight unit.")

class SupplierInfo(BaseModel):
    supplier_name: str
    contact_email: Optional[EmailStr] = None # Using EmailStr for email format validation

class ProductCatalogPydantic(BaseModel):
    product_id: str = Field(description="Unique identifier for the product.", pattern="^[A-Z0-9]{5,10}$")
    product_name: str = Field(description="Name of the product.")
    price: float = Field(description="Price of the product.", ge=0.01, le=10000.00) # ge = minimum, le = maximum
    in_stock: bool = Field(default=True, description="Availability status of the product.")
    tags: List[str] = Field(description="Keywords for the product.", min_length=1, max_length=5) # min_items/max_items for Pydantic v2 on Field are min_length/max_length
    category: Literal["Electronics", "Apparel", "Home Goods", "Books"] = Field(description="Product category.")
    dimensions_or_weight: Union[Dimensions, Weight] = Field(description="Either physical dimensions or weight of the product.")
    supplier: SupplierInfo = Field(description="Information about the product supplier.")

# To handle the $defs part correctly and have a clean top-level schema,
# it's good practice to ensure all model definitions are available.
# Pydantic's schema generation will handle creating the definitions section.

In [ ]:
# Generate the JSON schema from the Pydantic model
pydantic_generated_schema = ProductCatalogPydantic.model_json_schema()

# Print the generated schema (pretty printed)
# import json # Already imported earlier
print("JSON Schema generated from Pydantic model:")
print(json.dumps(pydantic_generated_schema, indent=2))

# You can also save it or compare it with my_schema if needed
# print("\n\nManually defined schema (for comparison):")
# print(json.dumps(my_schema, indent=2))

## Testing Union Types (anyOf) with Specific Prompts

The `dimensions_or_weight` field in our schema is a union type (defined using `anyOf` in JSON Schema or `Union` in Pydantic). We can test how the model responds when prompted for information that should fit one part of the union or the other.

In [ ]:
prompt_for_dimensions = "Describe a product that is a piece of furniture, like a small bookshelf. Please include its physical dimensions."

print(f"Prompting for product with dimensions:\n{prompt_for_dimensions}\n")

try:
    response_dimensions = client.models.generate_content(
        model='gemini-1.5-flash',
        contents=prompt_for_dimensions,
        config={
            'response_mime_type': 'application/json',
            'response_schema': my_schema # Using the manually defined schema
        },
    )
    if response_dimensions and response_dimensions.parsed:
        print("Response for 'dimensions' prompt:")
        print(json.dumps(response_dimensions.parsed, indent=2))
    else:
        print("Received no valid parsed data for the dimensions prompt.")
        if response_dimensions:
             print(f"Raw response text: {response_dimensions.text}")
except Exception as e:
    print(f"An error occurred processing dimensions prompt: {e}")

In [ ]:
prompt_for_weight = "Describe a product that is a small, dense item, like a calibration weight for a scale. Please specify its weight."

print(f"\nPrompting for product with weight:\n{prompt_for_weight}\n")

try:
    response_weight = client.models.generate_content(
        model='gemini-1.5-flash',
        contents=prompt_for_weight,
        config={
            'response_mime_type': 'application/json',
            'response_schema': my_schema # Using the manually defined schema
        },
    )
    if response_weight and response_weight.parsed:
        print("Response for 'weight' prompt:")
        print(json.dumps(response_weight.parsed, indent=2))
    else:
        print("Received no valid parsed data for the weight prompt.")
        if response_weight:
            print(f"Raw response text: {response_weight.text}")
except Exception as e:
    print(f"An error occurred processing weight prompt: {e}")

---
Now, let's see the original generic prompt's output again with the full schema.

In [ ]:
import os
from google import genai

# Ensure the client is initialized.
# If running this cell independently, you might need to uncomment the next line
# or ensure GEMINI_API_KEY is set in your environment.
# client = genai.Client(api_key=os.getenv("GEMINI_API_KEY")) 

# If the client is already defined in a previous cell (e.g. from Pydantic example), 
# this code will assume it's available.

prompt_json = "Generate a sample product listing for a new high-tech drone, be creative with the description and features."

try:
    response_json_schema = client.models.generate_content(
        model='gemini-1.5-flash', # Using a model that supports JSON schema well
        contents=prompt_json,
        config={
            'response_mime_type': 'application/json',
            'response_schema': my_schema  # Use the schema defined in the previous cell
        },
    )
except Exception as e:
    print(f"An error occurred: {e}")
    response_json_schema = None # Ensure the variable exists even if the call fails

In [ ]:
import json

if response_json_schema:
    # The .parsed attribute will contain the Python dict parsed from the JSON response
    parsed_product_data = response_json_schema.parsed
    print(json.dumps(parsed_product_data, indent=2))
else:
    print("No response to display as the API call might have failed.")

# You can also directly access parts of the parsed data if needed:
# if response_json_schema and response_json_schema.parsed:
#     print(f"Product Name: {response_json_schema.parsed.get('product_name')}")